# Wholesale data preprocessing
도매 데이터 전처리

In [2]:
!pip install sklearn

  Using cached sklearn-0.0.tar.gz (1.1 kB)
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
     ---------------------------------------- 7.4/7.4 MB 5.8 MB/s eta 0:00:00
     -------------------------------------- 307.0/307.0 kB 9.3 MB/s eta 0:00:00
     ---------------------------------------- 36.9/36.9 MB 8.8 MB/s eta 0:00:00
Using legacy 'setup.py install' for sklearn, since package 'wheel' is not installed.
  Running setup.py install for sklearn: started
  Running setup.py install for sklearn: finished with status 'done'


In [47]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder

In [48]:
ws = pd.read_csv('wholesale_2022-06-23.csv', index_col=0) # 1년치 데이터

In [49]:
ws

,item_name,item_code,kind_name,kind_code,rank,rank_code,unit,day1,dpr1
0,쌀,111,일반계(20kg),1,상품,4,20kg,2022-06-23,"44,500"
1,쌀,111,일반계(20kg),1,중품,5,20kg,2022-06-23,"41,000"
2,찹쌀,112,일반계(40kg),1,중품,5,40kg,2022-06-23,"97,800"
3,콩,141,흰 콩(국산)(40kg),1,상품,4,40kg,2022-06-23,"240,000"
4,콩,141,흰 콩(수입)(35kg),3,중품,5,35kg,2022-06-23,"122,000"
...,...,...,...,...,...,...,...,...,...
44564,건오징어,640,건오징어(20마리),0,중품,5,20마리,2021-06-24,"77,000"
44565,김,641,마른김(1속),0,중품,5,1속,2021-06-24,"6,000"
44566,건미역,642,건미역(1kg),0,상품,4,1kg,2021-06-24,"10,000"
44567,전복,653,전복(1kg),0,중품,5,1kg,2021-06-24,"33,000"


In [50]:
ws[:14] # 오후 7시 이후에 수집하는 것으로..

,item_name,item_code,kind_name,kind_code,rank,rank_code,unit,day1,dpr1
0,쌀,111,일반계(20kg),1,상품,4,20kg,2022-06-23,"44,500"
1,쌀,111,일반계(20kg),1,중품,5,20kg,2022-06-23,"41,000"
2,찹쌀,112,일반계(40kg),1,중품,5,40kg,2022-06-23,"97,800"
3,콩,141,흰 콩(국산)(40kg),1,상품,4,40kg,2022-06-23,"240,000"
4,콩,141,흰 콩(수입)(35kg),3,중품,5,35kg,2022-06-23,"122,000"
5,팥,142,붉은 팥(국산)(40kg),0,상품,4,40kg,2022-06-23,"360,000"
6,팥,142,붉은 팥(수입)(40kg),1,중품,5,40kg,2022-06-23,"266,000"
7,녹두,143,국산(40kg),0,상품,4,40kg,2022-06-23,"736,000"
8,녹두,143,수입(40kg),1,중품,5,40kg,2022-06-23,"216,000"
9,메밀,144,메밀(수입)(1kg),1,중품,5,1kg,2022-06-23,"4,500"


## 1 . EDA

### 1-1. 일자별 항목 갯수의 변화
거의 일정하긴 한데 변화가 있긴 해서 새로운 항목이 들어올 가능성을 열어둬야할 듯.<br>
-> 에러 나면 그때 고치자 호호

In [51]:
ws.groupby(['day1']).count()

,item_name,item_code,kind_name,kind_code,rank,rank_code,unit,dpr1
day1,,,,,,,,
2021-06-24,136,136,136,136,136,136,136,136
2021-06-25,136,136,136,136,136,136,136,136
2021-06-26,134,134,134,134,134,134,134,134
2021-06-28,134,134,134,134,134,134,134,134
2021-06-29,138,138,138,138,138,138,138,138
...,...,...,...,...,...,...,...,...
2022-06-18,134,134,134,134,134,134,134,134
2022-06-20,139,139,139,139,139,139,139,139
2022-06-21,139,139,139,139,139,139,139,139


### 1-2. 품목 별 카운트
카운트가 많을수록 한 품목 내에서 세분화된 정보가 제공됨을 의미

In [52]:
ws.groupby(['item_name']).count()

,item_code,kind_name,kind_code,rank,rank_code,unit,day1,dpr1
item_name,,,,,,,,
갈치,624,624,624,624,624,624,624,624
감귤,472,472,472,472,472,472,472,472
감자,716,716,716,716,716,716,716,716
건고추,1276,1276,1276,1276,1276,1276,1276,1276
건멸치,312,312,312,312,312,312,312,312
...,...,...,...,...,...,...,...,...
포도,900,900,900,900,900,900,900,900
풋고추,2012,2012,2012,2012,2012,2012,2012,2012
피마늘,1152,1152,1152,1152,1152,1152,1152,1152


In [53]:
ws[(ws['day1']=='2022-06-23')&(ws['item_name']=='풋고추')] # 풋고추는 무려 8가지...!

,item_name,item_code,kind_name,kind_code,rank,rank_code,unit,day1,dpr1
4457,풋고추,242,풋고추(10kg),0,상품,4,10kg,2022-06-23,"74,500"
4458,풋고추,242,풋고추(10kg),0,중품,5,10kg,2022-06-23,"66,500"
4459,풋고추,242,꽈리고추(4kg),2,상품,4,4kg,2022-06-23,"33,200"
4460,풋고추,242,꽈리고추(4kg),2,중품,5,4kg,2022-06-23,"28,200"
4461,풋고추,242,청양고추(10kg),3,상품,4,10kg,2022-06-23,"74,700"
4462,풋고추,242,청양고추(10kg),3,중품,5,10kg,2022-06-23,"68,200"
4463,풋고추,242,오이맛고추(10kg),4,상품,4,10kg,2022-06-23,"51,000"
4464,풋고추,242,오이맛고추(10kg),4,중품,5,10kg,2022-06-23,"48,500"


In [54]:
ws['item_code']

0        111
1        111
2        112
3        141
4        141
        ... 
44564    640
44565    641
44566    642
44567    653
44568    654
Name: item_code, Length: 44569, dtype: int64

### 1-3. rank의 종류

In [55]:
ws['rank'].value_counts() # M과 S과 -> 귤의 size 등급

중품    24200
상품    19897
S과      236
M과      236
Name: rank, dtype: int64

## 2 . Preprocessing

### 2-1. 가격정보가 들어오지 않은 것 NaN 처리

In [56]:
ws['dpr1'] = ws['dpr1'].replace("-", np.NaN)

### 2-2. 사용자가 검색하는 품목명이 될 kind_name_new를 생성
#### 2-2-1. func.make_product_name 생성

In [57]:
## ver.1
def make_product_name(item_name, kind_name):
    if item_name in kind_name: # item name이 kind name안에 있을 경우            
        result = kind_name # 그냥 kind name 그대로. ex. 콩,팥
    else:
        result = item_name + "(" + kind_name.split("(")[0] + ")(" + kind_name.split("(")[1] # ex. 녹두,감자,고구마
    return result

In [58]:
ws['kind_name_new'] = ws.apply(lambda x: make_product_name(x['item_name'], x['kind_name']), axis=1)

make_product_name에 의해 제대로 변환되었는지 확인

In [59]:
 ## 별도처리: 깐마늘, 피마늘 모두! -> 얘네만 kind_name에 있는 괄호의 갯수가 다름
ws['kind_name_new'].value_counts()

깻잎(2kg)            624
대파(1kg)            624
오이(취청)(50개)        624
애호박(20개)           624
호박(쥬키니)(10kg)      624
                  ... 
햇양파(15kg)           32
쌀(햇일반계)(20kg)       26
메밀(수입)(1kg)         18
건고추(햇산양건)(30kg)     14
건고추(햇산화건)(30kg)     14
Name: kind_name_new, Length: 116, dtype: int64

In [60]:
ws[ws['item_name'].str.contains('깐마늘')][['item_name', 'kind_name']].value_counts()
# kind_name_new를 깐마늘(국산)(대서)(20kg), 햇깐마늘(국산)(남도)(20kg)로 바꿔주고 싶음

item_name  kind_name     
깐마늘(국산)    깐마늘(대서)(20kg)     624
           깐마늘(남도)(20kg)     622
           햇깐마늘(남도)(20kg)     42
           햇깐마늘(대서)(20kg)     42
dtype: int64

In [61]:
ws[ws['item_name'].str.contains('피마늘')][['item_name', 'kind_name']].value_counts()
# kind_name_new를 피마늘(난지)(대서)(10kg) 로 바꿔주고 싶음
# cf. 난지,한지 = cool or warm, 대서,남도 = region 

item_name  kind_name   
피마늘        난지(대서)(10kg)    624
           한지(10kg)        462
           난지(남도)(10kg)     66
dtype: int64

#### 2-2-1. func.make_product_name2 생성
마늘 데이터를 처리해주기 위해 

In [62]:
## ver.2
def make_product_name2(item_name, kind_name):

    if "깐마늘" in item_name: # ok
        result = kind_name.split("(")[0] + '(국산)(' + kind_name.split("(")[1] + '('+kind_name.split("(")[2]
        
    elif "피마늘" in item_name:
        result = '피마늘(' + kind_name[:2] + ")" + kind_name[2:]

    else:
        if item_name in kind_name: # item name이 kind name안에 있을 경우 
            result = kind_name # 그냥 kind name 그대로
            
        else:
            result = item_name + "(" + kind_name.split("(")[0] + ")(" + kind_name.split("(")[1]

    return result

In [63]:
ws['kind_name_new'] = ws.apply(lambda x: make_product_name2(x['item_name'], x['kind_name']), axis=1)

In [64]:
ws['kind_name_new'].value_counts()[100:] # 체크 완료

복숭아(백도)(4.5kg)        102
배추(가을)(10kg)           96
무(가을)(20kg)            96
오렌지(발렌시아 미국)(18kg)     92
감자(대지마)(20kg)          92
배(원황)(15kg)            82
무(봄)(20kg)             80
피마늘(난지)(남도)(10kg)      66
사과(쓰가루)(10kg)          64
햇깐마늘(국산)(대서)(20kg)     42
햇깐마늘(국산)(남도)(20kg)     42
햇양파(15kg)              32
쌀(햇일반계)(20kg)          26
메밀(수입)(1kg)            18
건고추(햇산양건)(30kg)        14
건고추(햇산화건)(30kg)        14
Name: kind_name_new, dtype: int64

### 2-3. p_code(=제품 코드) 생성
p_code의 구성
- 땅에서 나는거: A(Agricultural), 바다에서 나는 거: S(Seafood), 축산: L(Live Stock)
- Wholesale이니까 W
- 숫자 앞을 막아줄 A padding
- kind_name_new + rank -> label encoding -> zfill(5)

#### 2-3-1. category_cd 생성
- 100:식량작물, 200:채소류, 300:특용작물, 400:과일류 -> AW
- 500:축산물 -> LW
- 600:수산물 -> SW

In [65]:
ws['category_cd'] = ws['item_code'].map(
    lambda x: "LW" if 500<=x<600 else (
        "SW" if x>=600
        else "AW"
    )
)

서울에서 가축을 못키워서 축산물 거래가 없는 것으로 추정했으나, <br>
몇년 전부터 축산데이터만 제공을 안해서 따로 가져와야함 -> 일단 skip! 갖고 있는 것만 가지고 잘 해보자

In [66]:
ws['category_cd'].value_counts()

AW    39738
SW     4831
Name: category_cd, dtype: int64

#### 2-3-2. kind_name_rank 생성
kind_name_rank = kind_name + rank<br>
제품명과 상품 등급을 묶어서 unique한 조합을 만들기 위함

In [67]:
ws['kind_name_rank'] = ws['kind_name_new'] + ws['rank']

#### 2-3-3. Label encoding kind_name_rank 
kind_name_rank라는 문자열의 조합을 Label Encoder를 통해 int로 변환

In [68]:
from sklearn.preprocessing import LabelEncoder

In [69]:
encoder = LabelEncoder()
ws['kind_name_rank_label'] = encoder.fit_transform(ws['kind_name_rank'])

if label encoding 결과가 34 -> A00034로 만들어주기 위해 다음의 코드 실행.

In [70]:
ws['kind_name_rank_label'] = ws['kind_name_rank_label'].map(lambda x: "A"+str(x).zfill(5))

In [71]:
ws['kind_name_rank_label']

0        A00115
1        A00116
2        A00153
3        A00198
4        A00200
          ...  
44564    A00020
44565    A00053
44566    A00019
44567    A00140
44568    A00107
Name: kind_name_rank_label, Length: 44569, dtype: object

#### 2-3-4. p_code 생성
with category_cd + kind_name_rank_label

In [72]:
ws['p_code'] = ws['category_cd'] + "_" + ws['kind_name_rank_label']

In [73]:
ws.head()

,item_name,item_code,kind_name,kind_code,rank,rank_code,unit,day1,dpr1,kind_name_new,category_cd,kind_name_rank,kind_name_rank_label,p_code
0,쌀,111,일반계(20kg),1,상품,4,20kg,2022-06-23,"44,500",쌀(일반계)(20kg),AW,쌀(일반계)(20kg)상품,A00115,AW_A00115
1,쌀,111,일반계(20kg),1,중품,5,20kg,2022-06-23,"41,000",쌀(일반계)(20kg),AW,쌀(일반계)(20kg)중품,A00116,AW_A00116
2,찹쌀,112,일반계(40kg),1,중품,5,40kg,2022-06-23,"97,800",찹쌀(일반계)(40kg),AW,찹쌀(일반계)(40kg)중품,A00153,AW_A00153
3,콩,141,흰 콩(국산)(40kg),1,상품,4,40kg,2022-06-23,"240,000",흰 콩(국산)(40kg),AW,흰 콩(국산)(40kg)상품,A00198,AW_A00198
4,콩,141,흰 콩(수입)(35kg),3,중품,5,35kg,2022-06-23,"122,000",흰 콩(수입)(35kg),AW,흰 콩(수입)(35kg)중품,A00200,AW_A00200


## 3 . Column rename & Feature selection

In [74]:
ws.head()

,item_name,item_code,kind_name,kind_code,rank,rank_code,unit,day1,dpr1,kind_name_new,category_cd,kind_name_rank,kind_name_rank_label,p_code
0,쌀,111,일반계(20kg),1,상품,4,20kg,2022-06-23,"44,500",쌀(일반계)(20kg),AW,쌀(일반계)(20kg)상품,A00115,AW_A00115
1,쌀,111,일반계(20kg),1,중품,5,20kg,2022-06-23,"41,000",쌀(일반계)(20kg),AW,쌀(일반계)(20kg)중품,A00116,AW_A00116
2,찹쌀,112,일반계(40kg),1,중품,5,40kg,2022-06-23,"97,800",찹쌀(일반계)(40kg),AW,찹쌀(일반계)(40kg)중품,A00153,AW_A00153
3,콩,141,흰 콩(국산)(40kg),1,상품,4,40kg,2022-06-23,"240,000",흰 콩(국산)(40kg),AW,흰 콩(국산)(40kg)상품,A00198,AW_A00198
4,콩,141,흰 콩(수입)(35kg),3,중품,5,35kg,2022-06-23,"122,000",흰 콩(수입)(35kg),AW,흰 콩(수입)(35kg)중품,A00200,AW_A00200


In [75]:
ws_table = ws.rename(columns = {'item_name':'p_group', 'day1':'date', 'dpr1':'price'})
ws_table['category_code'] = ws_table['p_code']
ws_table = ws_table[['category_code', 'p_code', 'p_group', 'kind_name_new', 'rank', 'unit', 'price', 'date']]

In [76]:
ws_table[:14]

,category_code,p_code,p_group,kind_name_new,rank,unit,price,date
0,AW_A00115,AW_A00115,쌀,쌀(일반계)(20kg),상품,20kg,"44,500",2022-06-23
1,AW_A00116,AW_A00116,쌀,쌀(일반계)(20kg),중품,20kg,"41,000",2022-06-23
2,AW_A00153,AW_A00153,찹쌀,찹쌀(일반계)(40kg),중품,40kg,"97,800",2022-06-23
3,AW_A00198,AW_A00198,콩,흰 콩(국산)(40kg),상품,40kg,"240,000",2022-06-23
4,AW_A00200,AW_A00200,콩,흰 콩(수입)(35kg),중품,35kg,"122,000",2022-06-23
5,AW_A00091,AW_A00091,팥,붉은 팥(국산)(40kg),상품,40kg,"360,000",2022-06-23
6,AW_A00092,AW_A00092,팥,붉은 팥(수입)(40kg),중품,40kg,"266,000",2022-06-23
7,AW_A00033,AW_A00033,녹두,녹두(국산)(40kg),상품,40kg,"736,000",2022-06-23
8,AW_A00034,AW_A00034,녹두,녹두(수입)(40kg),중품,40kg,"216,000",2022-06-23
9,AW_A00056,AW_A00056,메밀,메밀(수입)(1kg),중품,1kg,"4,500",2022-06-23


In [77]:
ws_table['category_code'] == ws_table['p_code']

0        True
1        True
2        True
3        True
4        True
         ... 
44564    True
44565    True
44566    True
44567    True
44568    True
Length: 44569, dtype: bool

In [78]:
ws_table

,category_code,p_code,p_group,kind_name_new,rank,unit,price,date
0,AW_A00115,AW_A00115,쌀,쌀(일반계)(20kg),상품,20kg,"44,500",2022-06-23
1,AW_A00116,AW_A00116,쌀,쌀(일반계)(20kg),중품,20kg,"41,000",2022-06-23
2,AW_A00153,AW_A00153,찹쌀,찹쌀(일반계)(40kg),중품,40kg,"97,800",2022-06-23
3,AW_A00198,AW_A00198,콩,흰 콩(국산)(40kg),상품,40kg,"240,000",2022-06-23
4,AW_A00200,AW_A00200,콩,흰 콩(수입)(35kg),중품,35kg,"122,000",2022-06-23
...,...,...,...,...,...,...,...,...
44564,SW_A00020,SW_A00020,건오징어,건오징어(20마리),중품,20마리,"77,000",2021-06-24
44565,SW_A00053,SW_A00053,김,마른김(1속),중품,1속,"6,000",2021-06-24
44566,SW_A00019,SW_A00019,건미역,건미역(1kg),상품,1kg,"10,000",2021-06-24
44567,SW_A00140,SW_A00140,전복,전복(1kg),중품,1kg,"33,000",2021-06-24


In [79]:
ws_table.to_csv('./result/wholesale_2022-06-23_preprocessed.csv', encoding = 'utf-8', index=False)